# Paso 1. Ecuación Unidimensional de Convección Lineal

\begin{equation}
\frac{\partial \psi}{\partial t} + c \frac{\partial \psi}{\partial x} = 0
\end{equation}

Lo que nos dice esta ecuación es que la derivada de una función con respecto al tiempo es directamente proporcial a la velocidad (donde la constante de proporcionalidad es $-c$).

\begin{equation}
\frac{\partial \psi}{\partial t} = -c \frac{\partial \psi}{\partial x}
\end{equation}

La solución analítica para esta ecuación diferencial es sencilla. Dada la condición inicial:
\begin{equation*}
\psi(x, 0) = f(x)
\end{equation*}

Digamos que $\psi(x, t) = f(x -ct)$ es una solución de la ecuación diferencial. Entonces:
\begin{eqnarray*}
\frac{\partial f}{\partial t} &=& -c f'(x-ct) \\
\frac{\partial f}{\partial x} &=& f'(x-ct) \\
\end{eqnarray*}
Reemplazando:
\begin{eqnarray*}
-c f'(x-ct) + c f'(x-ct) = 0
\end{eqnarray*}
Con lo cual observamos que es solución. 

Veamos un ejemplo.
Sea $f(x) = cos(x)$, entonces:
\begin{eqnarray*}
\frac{\partial f}{\partial t} &=& -c \sin(x-ct) \\
\frac{\partial f}{\partial x} &=& - \sin(x-ct) \\
\end{eqnarray*}

Reemplazando en la ecuación de convección:
\begin{equation}
-c \sin(x-ct) -c \sin(x-ct) = 0
\end{equation}

Lo que podemos observar es que al ser la solución la propia condición inicial con parámetro $x-ct$, lo único que se esta haciendo es un corrimiento en el espacio del perfil inicial de la función a lo largo del tiempo.

# Discretización, diferencias finitas


Para la discretización de este problema utilizaremos el equema numérico FT-BS, (Forward in Time, Backward in space). Como estandar diremos que el subindice $i$ corresponde a la posición en el espacio y $n$ para el subindice en el tiempo. Unilizando esta notación, tenemos para la ecuación diferencial su correspondiente discretización.

$$
\frac{\partial \psi}{\partial t} + c \frac{\partial \psi}{\partial x} = 0 \\
\frac{\psi(i)_{n+1} - \psi(i)_n }{\Delta t} +c \frac{\psi(i)_n - \psi(i-1)_n}{\Delta x} = 0
$$

Analizando la discretización podemos observar que tenemos todas las variables, $\psi(i)_n$ y $\psi(i-1)_n$ estan dadas por la condición inicial, $\Delta t$ y $\Delta x$ son el tamaño de paso que elijamos, por lo tanto solo nos resta el término desconocido $\psi(i)_{n+1}$, el cual podemos hayar resolviendo la ecuación discretizada. Así tenemos:

$$
\psi(i)_{n+1} = \psi(i)_n - c \frac{\Delta t}{\Delta x} \Big[\psi(i)_n - \psi(i-1)_n\Big]
$$

Con la expresión anterior podemos hayar el comportamiento del profile inicial en el tiempo.

Tomemos para nuestro ejemplo la función Gaussiana

$$f(x) = \frac{1} {\sqrt{2\pi \sigma^2}} \exp^{ -\frac{(x-\mu)^2}{2\sigma^2} }$$

Si tomamos $\mu = 0, \sigma^2 = 0.2$

$$f(x) = \frac{1} {\sigma\sqrt{2\pi}} \exp^{ -\frac{x^2}{2\sigma^2} }$$

In [10]:
import matplotlib.pyplot as plt
from matplotlib import animation
import numpy as np;

# How much we want to advance in time?
# at 30 fps if we want simulate 10 seconds => 30*10
t_seconds = 30 * 7;

# The time step size 
dt = 0.005;

# x velocity
c = 10.0;

# spatial partitions
x_partitions = 100;

data = np.linspace(-2.0, 8.0, num=x_partitions, retstep=True);

x  = data[0];
dx = data[1];

# Set initial profile
miu = 0;
sigmaSqrt = 0.2;

u = (1.0 / np.sqrt(2*np.pi*sigmaSqrt)) * np.exp( -(x-miu)**2/(2*sigmaSqrt) );

fig = plt.figure();
fig.suptitle(r'$\frac{\partial u}{\partial t} + c \frac{\partial u}{\partial x} = 0$', fontsize=20, fontweight='bold');
ax = plt.axes(xlim=(-2.0, 8.0), ylim=(-0.25, 1.0));
profile, = ax.plot([], [], lw=2);
realSolution, = ax.plot([], [], lw=2);

def init():
    profile.set_data([], []);
    return profile, realSolution,

def update(n): 
    
    if(n == 0):
        profile.set_data(x, u);
        realSolution.set_data(x, u);
        return profile, realSolution,
    
    #store the actual state of the array
    u_n = u;
    
    #update array
    u[1:] = u_n[1:] - (c*(dt/dx)) * (u_n[1:] - np.roll(u_n, 1)[1:]);
                    
    profile.set_data(x, u);
    
    temp = (1.0 / np.sqrt(2*np.pi*sigmaSqrt)) * np.exp( -((x-c*dt*n)-miu)**2/(2*sigmaSqrt) );
    realSolution.set_data(x, temp);
    
    return profile, realSolution,

# The following lines works only on a machine with ffmpeg configured
anim = animation.FuncAnimation(fig, update, init_func=init, frames=t_seconds, blit=True);
anim.save('1D_Linear_Convection_Equation_01.mp4', fps=30, writer="ffmpeg", codec="libx264");
#plt.show(fig);
plt.close(fig);

In [11]:
# You must have configured ffmpeg in your machine to run the code below
from IPython.display import HTML
video = open("1D_Linear_Convection_Equation_01.mp4", "rb").read()
video_encoded = video.encode("base64")
video_tag = '<video controls alt="test" src="data:video/x-m4v;base64,{0}">'.format(video_encoded)
HTML(video_tag)

Como se observa en la simulación, el profile inicial tiene un decaimiento que podria mal interpretarse como una viscosidad, esto es debido al error numérico que introducen los cálculos de la computadora, generando así la viscocidad artificial